In [2]:
import json
import pickle

In [3]:
# Original
with open("/u/home/j/jiayini/project-sriram/RHE_project/results/original_result_8.json", "r") as file:
    data = json.load(file)

parsed_data = {}

for key, value in data.items():
    idx = int(key.split("/")[-1].split(".")[0])
    sigma_est_list = []
    sigma_err_list = []
    
    i = 0
    while f'sigma_est_{i}' in value:
        sigma_est_list.append(value[f'sigma_est_{i}'])
        sigma_err_list.append(value[f'sigma_err_{i}'])
        i += 1

    parsed_data[idx] = [sigma_est_list, sigma_err_list]

for idx, values in sorted(parsed_data.items()):
    print(f"{idx} : {values}")

0 : [[0.0380205, 0.0274348, 0.0587523, 0.0128947, 0.0459043, 0.0189773, 0.0061354, 0.0374215], [0.0148969, 0.0145259, 0.013197, 0.0127906, 0.0139662, 0.0125244, 0.0148579, 0.0135782]]
1 : [[0.0379041, 0.0322957, 0.0375021, 0.0323146, 0.0348979, 0.0321376, 0.0439422, 0.0212363], [0.0132358, 0.012794, 0.0129908, 0.013581, 0.0135797, 0.0131711, 0.0131676, 0.0132133]]
2 : [[0.0241348, 0.0388243, 0.0353498, 0.0228176, 0.0282551, 0.0538214, 0.0133043, 0.0386014], [0.014524, 0.0137977, 0.0139937, 0.0139609, 0.0126978, 0.014259, 0.0120819, 0.0135001]]
3 : [[0.0229964, 0.0562998, 0.0424698, 0.0249466, 0.017463, 0.00853183, 0.0267356, 0.054944], [0.0126757, 0.0137666, 0.0126771, 0.0143953, 0.0123228, 0.0142382, 0.0139611, 0.0127096]]
4 : [[0.0301301, 0.0128847, 0.0336403, 0.0267903, 0.0361285, 0.0160788, 0.0534453, 0.039311], [0.0129834, 0.0133223, 0.012882, 0.0128637, 0.0118126, 0.0125545, 0.0125779, 0.0111416]]
5 : [[0.0266141, 0.0518203, 0.0350273, 0.0248734, 0.0136085, 0.0503221, 0.0379438, 

In [5]:
file_path = "/u/home/j/jiayini/project-sriram/RHE_project/python_rhe.pkl"
with open(file_path, "rb") as file:
    content = pickle.load(file)

print(content)
# TODO: Parse
# TODO: multiple runs + plotting

{'/u/home/j/jiayini/project-sriram/RHE_project/data/pheno/bin_1/1.phen': {'sigma_ests_total': array([0.03200764, 0.01770768, 0.02884342, 0.04388027, 0.02604349,
       0.02038837, 0.03856524, 0.03474782, 0.747994  ]), 'sig_errs': [0.01380795811071219, 0.013988756478167198, 0.013604864736457986, 0.011229943767525272, 0.0147056288032917, 0.013301387965601475, 0.013351993685053866, 0.012213313452058295, 0.021448031889042422], 'h2_total': array([0.03232514, 0.01788333, 0.02912953, 0.04431554, 0.02630183,
       0.02059061, 0.03894779, 0.0350925 , 0.75541373, 0.24458627]), 'h2_errs': [0.01394492620713512, 0.014127518004728612, 0.013739818247332175, 0.01134133924022597, 0.014851501347786646, 0.013433331137417598, 0.013484438840509132, 0.012334463464328914, 0.02166078572826355, 0.02166078572826357], 'enrichment_total': array([2.58663961e-07, 1.43124368e-07, 2.33186857e-07, 3.54419679e-07,
       2.10377627e-07, 1.64733285e-07, 3.11692799e-07, 2.80765879e-07]), 'enrichment_errs': [1.1114636654